# Introdução

Este projeto tem como objetivo coletar, armazenar e analisar dados históricos de condições climáticas provenientes de diferentes cidades. Utilizando uma base de dados extraída da **HG Brasil** e salva em banco usando **MongoDB**, a aplicação deverá realizar testes com diferentes algoritmos de *machine learning* para registrar seus resultados e permitir uma comparação de eficiência. Os dados coletados incluem temperatura, umidade, descrição do clima, velocidade do vento, entre outros parâmetros relevantes para estudos meteorológicos e análises estatísticas. 

### Inicialização

Para carregar os módulos necessários, execute a célula abaixo para iniciar as bibliotecas usadas para o funcionamento do projeto.

In [ ]:
import json
import pandas as pd

from src.db_config import get_data
from src.config import BACKUP_DB, BACKUP_DIR, DB_TABLE_WEATHER

*A base usada durante o treinamento será carregada na célula abaixo, precisando ser inicializada antes de executar os demais pontos.*

In [ ]:
data = get_data(DB_TABLE_WEATHER)

*O código abaixo permite realizar a validação dos dados duplicados, usado para o ajuste de base.*

In [ ]:
contagem = {}
duplicados = []

for item in data:
    chave = (item["results"]["city"], item["results"]["date"], item["results"]["time"])
    contagem[chave] = contagem.get(chave, 0) + 1

resultado = [list(chave) + [quantidade] for chave, quantidade in contagem.items()]

for item_resultado in [filtro_resultado for filtro_resultado in resultado if filtro_resultado[3] > 1]:
    for item in [filtro for filtro in data if filtro["results"]["city"] == item_resultado[0] and filtro["results"]["date"] == item_resultado[1] and filtro["results"]["time"] == item_resultado[2]]:
        duplicados.append({"Cidade": item["results"]["city"], "Data": item["results"]["date"], "Hora": item["results"]["time"], "ID": item["_id"]})

pd.DataFrame(duplicados)

*O bloco abaixo permite a validação parcial dos dados, podendo ser usado para garantir que eles estão sendo salvos corretamente.*

In [ ]:
for item in [filtro for filtro in data if filtro["results"]["date"] == "01/06/2025"]:
    print(f"{item["results"]["city"]} - {item["results"]["date"]} {item["results"]["time"]}: {item["results"]["temp"]}C° ({item["results"]["description"]})")

## Backup

Caso seja necessário realizar um backup das informações salvas em arquivos JSON poderá ser executado o comando a seguir, porém garantindo que as informações abaixo estão corretas no *.env*.
- **BACKUP_DB** - Ajustar a variável para permitir a cópia
- **BACKUP_DIR** - Incluir um diretório válido

In [ ]:
if BACKUP_DB:
   for item in data:
      with open(f"{BACKUP_DIR}/{item["_id"]}.json", "w") as arquivo:
         del item["_id"]

         json.dump(item, arquivo)